In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.onnx
import onnx
import onnxruntime
import numpy as np

In [2]:
# 함수 선언

from NN import Encoder
from NN import Attention
from NN import Decoder

In [3]:
# encoder 함수 불러오기

encoder = torch.load("encoder.pt", weights_only=False, map_location="cpu")
encoder.eval()

print(encoder)

Encoder(
  (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
  (rnn): LSTM(64, 76, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (dropout): Dropout(p=0.1, inplace=False)
)


In [4]:
# encoder 함수 저장

x = torch.randn(1,100,64) # dummy 입력값 0번째 : 소리갯수, 1번째 : 소리길이, 2번째 : 소리벡터차원 
dynamic_axes = {'x' : {1 : 'f'}, 'y' : {1 : 'f'}} #소리 길이만 가변 길이로 정하겠습니다 (소리 하나씩만 처리 가능한 onnx 신경망으로 저장)

torch.onnx.export(
    encoder,
    x,
    "encoder.onnx",
    export_params=True, #W, b값
    opset_version=20,
    do_constant_folding=True, #상수 저장 방식
    input_names=['x'], #onnx내 입력값 이름
    output_names=['y','h','c'], #onnx내 출력값 이름
    dynamic_axes=dynamic_axes, #가변길이 차원 지정
)

C:\Users\Administrator\anaconda3\Lib\site-packages\torch\onnx\symbolic_opset9.py:4244: UserWarning: Exporting a model to ONNX with a batch_size other than 1, with a variable length with LSTM can cause an error when running the ONNX model with a different batch size. Make sure to save the model with a batch size of 1, or define the initial states (h0/c0) as inputs of the model. 
  warnings.warn(


In [5]:
onnx_F = onnxruntime.InferenceSession("encoder.onnx", providers=["CPUExecutionProvider"])
np_x = np.random.randn(1,150,64).astype(np.float32)
x = torch.tensor(np_x, dtype = torch.float)

y, h, c = encoder(x) # torch 결과값

inputs = {onnx_F.get_inputs()[0].name : np_x} 
outs = onnx_F.run(None, inputs) 
onnx_y = outs[0]
onnx_h = outs[1]
onnx_c = outs[2] # onnx

np.testing.assert_allclose(y.detach().numpy(), onnx_y, rtol=1e-03, atol=1e-05)
np.testing.assert_allclose(h.detach().numpy(), onnx_h, rtol=1e-03, atol=1e-05)
np.testing.assert_allclose(c.detach().numpy(), onnx_c, rtol=1e-03, atol=1e-05)

In [6]:
# decoder 함수 불러오기

decoder = torch.load("decoder.pt", weights_only=False, map_location='cpu')
decoder.eval()

decoder.forward = decoder.forward_cal

print(decoder)

Decoder(
  (embedding): Embedding(11181, 76, padding_idx=0)
  (attention): Attention(
    (U): Linear(in_features=152, out_features=152, bias=True)
    (W): Linear(in_features=152, out_features=152, bias=True)
    (V): Linear(in_features=152, out_features=1, bias=True)
  )
  (rnn): LSTM(228, 76, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (f): Linear(in_features=380, out_features=11181, bias=True)
)


In [7]:
# decoder 함수 저장

x = torch.randint(0,11181,size = (1,1)).type(torch.long) # 더미 입력값 지정
dynamic_axes = {'encoder_y' : {1 : 'f'}} 
#encoder 소리 길이만 가변적, 문장 하나만 처리하기 때문에 비가변적, decoder에 단어 넣을 때는 단어 하나만 넣기 때문에 비가변적

torch.onnx.export(
    decoder,
    (x, h, c, y),
    "decoder.onnx",
    export_params=True,
    opset_version=20,
    do_constant_folding=True,
    input_names = ['x','prev_h','prev_c','encoder_y'],
    output_names = ['y','h','c'],
    dynamic_axes=dynamic_axes
)